In [1]:
%matplotlib inline

# path hack for relative import in jupyter notebook
import os
import sys

# LIBRARY GLOBAL MODS
CELLTYPES = os.path.dirname(os.path.abspath(''))
sys.path.append(CELLTYPES)

In [2]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import pandas as pd
import umap
import time

sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})

from utils.file_io import RUNS_FOLDER

NOTEBOOK_OUTDIR = RUNS_FOLDER + os.sep + 'explore' + os.sep + 'nb_streamlined'

Appended to sys path I:\Development\Repositories\biomodels\celltypes


## Local utils

In [3]:
from multicell.unsupervised_helper import \
    plotly_express_embedding, generate_control_data, plot_given_multicell, make_dimreduce_object

from singlecell.singlecell_linalg import sorted_eig

## Settings: dimension reduction

In [4]:
# these set the defaults for modifications introduced in main
REDUCER_SEED = 0
REDUCER_COMPONENTS = 2
VALID_REDUCERS = ['umap', 'tsne', 'pca']
REDUCERS_TO_USE = ['umap']
assert REDUCERS_TO_USE == ['umap']  # for now, extend later

# see defaults: https://umap-learn.readthedocs.io/en/latest/api.html
"""AP_KWARGS = {
    'random_state': REDUCER_SEED,
    'n_components': REDUCER_COMPONENTS,
    'metric': 'euclidean',
    'init': 'spectral',
    'unique': False,
    'n_neighbors': 15,
    'min_dist': 0.1,
    'spread': 1.0,
}"""
UMAP_KWARGS = {
    'random_state': REDUCER_SEED,
    'n_components': REDUCER_COMPONENTS,
    'metric': 'euclidean',
    'init': 'spectral',
    'unique': False,
    'n_neighbors': 60,
    'min_dist': 0.1,
    'spread': 1.0,
}
TSNE_KWARGS = {
    'random_state': REDUCER_SEED,
    'n_components': REDUCER_COMPONENTS,
    'metric': 'euclidean',
    'init': 'random',
    'perplexity': 30.0,
}
PCA_KWARGS = {
    'n_components': REDUCER_COMPONENTS,
}

## Meanfield version of `make_dimreduce_object`

In [5]:
from singlecell.singlecell_functions import label_to_state, state_to_label

def LOCAL_make_dimreduce_object_meanfield(
    data_subdict, flag_control=False, nsubsample=None,
    use_01=True, jitter_scale=0.0, reducers=REDUCERS_TO_USE,
    umap_kwargs=UMAP_KWARGS, pca_kwargs=PCA_KWARGS, tsne_kwargs=TSNE_KWARGS, 
    step=None, remove_symmetry_ordering=True):
    """
    :param data_subdict:
    :param flag_control:
    :param nsubsample:
    :param use_01:
    :param jitter_scale:
    :param umap_kwargs:
    :param pca_kwargs:
    :param tsne_kwargs:
    :param step:  step of the simulation e.g. 'X_aggregate_7.npz'
        if None, then use 'X_aggregate.npz' (corresponds to last step)
    :return:
    """
    if flag_control:
        data_subdict['algos'] = {}
        #X = data_subdict['data']
        if nsubsample is not None:
            data_subdict['data'] = data_subdict['data'][0:nsubsample, :]
    else:
        manyruns_path = data_subdict['path']

        #smod = ''
        smod = '_last'  # '' is old style, '_last' is new style
        if step is not None:
            smod = '_%d' % step

        agg_dir = manyruns_path + os.sep + 'aggregate'
        fpath_state = agg_dir + os.sep + 'X_aggregate%s.npz' % smod
        fpath_energy = agg_dir + os.sep + 'X_energy%s.npz' % smod
        fpath_pickle = manyruns_path + os.sep + 'multicell_template.pkl'
        X = np.load(fpath_state)['arr_0'].T  # umap wants transpose
        X_energies = np.load(fpath_energy)['arr_0'].T  # umap wants transpose (?)
        
        with open(fpath_pickle, 'rb') as pickle_file:
            multicell_template = pickle.load(pickle_file)  # unpickling multicell object
            num_cells = multicell_template.num_cells
            num_genes = multicell_template.num_genes
            
        if remove_symmetry_ordering:
            assert nsubsample is None
            
            def transform_X_meanfield(X_orig):
                # Note: this should only be used when  'meanfield' interactions are used -- we discard specific orderings
                # 1) produce 'ordered' version of each lattice state
                # 2) keep only unique copies of the ordering
                num_runs, total_spins = X_orig.shape
                assert total_spins == num_cells * num_genes
                
                X_cells_as_cols = np.copy(X_orig)
                X_cells_as_cols = np.reshape(X_cells_as_cols, (-1, num_genes, num_cells), order='F')
                X_as_labels = np.zeros((num_runs, num_cells), dtype=int)
                X_meanfield = np.zeros((num_runs, total_spins), dtype=int)
                
                # A) create X_as_labels (cells as integers 0 to 2^N)
                for k in range(num_runs):
                    for c in range(num_cells):
                        cellstate = X_cells_as_cols[k, :, c]
                        X_as_labels[k, c] = state_to_label(cellstate)
                    
                    # Perform sorting (this requires meanfield case)
                    X_as_labels[k, :] = np.sort(X_as_labels[k, :])
                    
                    # Tranform original datase
                    for c in range(num_cells):
                        X_cells_as_cols[k, :, c] = label_to_state(X_as_labels[k, c], num_genes)

                    # Reshape to single axis: num_genes * num_cells
                    X_meanfield[k, :] = X_cells_as_cols[k, :, :].reshape(total_spins)
                            
                # C) identify duplicate runs
                # TODO later -- for now we will keep the runs as duplicates, umap will handle it naturally
                return X_meanfield
            
            #indices_to_keep = select_indices_to_keep()
            #X = X[indices_to_keep, :] 
            #X_energies = X[indices_to_keep, :] 
            X_new = transform_X_meanfield(X)
            X = X_new

        if nsubsample is not None:
            X = X[0:nsubsample, :]
            X_energies = X_energies[0:nsubsample, :]

        # store data and metadata in datasets object
        num_runs, total_spins = X.shape
        data_subdict['data'] = X
        data_subdict['index'] = list(range(num_runs))
        data_subdict['energies'] = X_energies
        data_subdict['num_runs'] = num_runs
        data_subdict['total_spins'] = total_spins
        data_subdict['multicell_template'] = multicell_template  # not needed? stored already
        data_subdict['algos'] = {}

    # binarization step needed for umap's binary metrics
    #  - convert +1, -1 to +1, 0
    if use_01:
        data_subdict['data'] = (1 + data_subdict['data']) / 2.0
        data_subdict['data'] = data_subdict['data'].astype(int)
        #X = (1 + X) / 2.0
        #X = X.astype(int)

    if jitter_scale > 0:
        # add gaussian noise to data with std=jitter_scale
        jitter = np.random.normal(0.0, jitter_scale, size=data_subdict['data'].shape)
        data_subdict['data'] = data_subdict['data'] + jitter

    # perform dimension reduction
    for algo in reducers:
        assert algo in VALID_REDUCERS
        data_subdict['algos'][algo] = {}

        t1 = time.time()
        if algo == 'umap':
            data_subdict['algos'][algo]['reducer'] = umap.UMAP(**umap_kwargs)
            data_subdict['algos'][algo]['reducer'].fit(data_subdict['data'])
            embedding = data_subdict['algos'][algo]['reducer'].transform(
                data_subdict['data']
            )
            data_subdict['algos'][algo]['embedding'] = embedding
        elif algo == 'pca':
            data_subdict['algos'][algo]['reducer'] = PCA(**pca_kwargs)
            embedding = data_subdict['algos'][algo]['reducer'].fit_transform(
                data_subdict['data']
            )
            data_subdict['algos'][algo]['embedding'] = embedding
        else:
            assert algo == 'tsne'
            data_subdict['algos'][algo]['reducer'] = TSNE(**tsne_kwargs)
            embedding = data_subdict['algos'][algo]['reducer'].fit_transform(
                data_subdict['data']
            )
            data_subdict['algos'][algo]['embedding'] = embedding
        print('Time to fit (%s): %.2f sec' % (algo, (time.time() - t1)))

    return data_subdict

### 0) Dataset path

In [6]:
# 0) load dataset
gamma = 1.0

manyruns_dirname = 'Wrandom0_gamma%.2f_10k_periodic_R1_p3_M100_machineEps' % gamma
#manyruns_dirname = 'Wmaze15_R5_gamma%.2f_10k_p3_M100' % gamma
#manyruns_dirname = 'Wrandom0_gamma%.2f_10k_fixedorder_p3_M4' % gamma
#manyruns_dirname = 'Wrandom0_gamma%.2f_10k_periodic_fixedorderV3_p3_M100' % gamma
#manyruns_dirname = 'Wrandom0_gamma%.2f_10k_periodic_fixedorderV3_p3_M100' % gamma
#manyruns_dirname = 'Wvary_s0randomInit_gamma%.2f_10k_periodic_fixedorderV3_p3_M100' % gamma
manyruns_path = RUNS_FOLDER + os.sep + 'multicell_manyruns' + os.sep + manyruns_dirname
    
# Step 0) load data
data_subdict = {'label': manyruns_dirname,
                'path': manyruns_path}

### 1) DImension reduction (store in data_subdict object)

In [7]:
use_01 = False
nsubsample = None
meanfield_variant = True

if meanfield_variant:
    dimereduce_fn = LOCAL_make_dimreduce_object_meanfield
else:
    dimereduce_fn = make_dimreduce_object
    
    
# 1) fill out data_subdict (dim reduce)
data_subdict = dimereduce_fn(
    data_subdict, 
    nsubsample=nsubsample, 
    flag_control=False,
    use_01=use_01, 
    jitter_scale=0.0,
    reducers=REDUCERS_TO_USE,
    umap_kwargs=UMAP_KWARGS, tsne_kwargs=TSNE_KWARGS, pca_kwargs=PCA_KWARGS,
    step=None)
#save_dimreduce_object(datasets[idx], fpath)  # save to file (joblib)

c:\users\matts\appdata\local\programs\python\python38\lib\site-packages\proplot\colors.py:976: ProPlotWarning: Shifting non-cyclic colormap 'acton_truncate'. Use cmap.set_cyclic(True) or Colormap(..., cyclic=True) to suppress this warning.
c:\users\matts\appdata\local\programs\python\python38\lib\site-packages\proplot\colors.py:976: ProPlotWarning: Shifting non-cyclic colormap 'Sunrise_truncate'. Use cmap.set_cyclic(True) or Colormap(..., cyclic=True) to suppress this warning.
c:\users\matts\appdata\local\programs\python\python38\lib\site-packages\proplot\colors.py:976: ProPlotWarning: Shifting non-cyclic colormap 'Sunset_truncate'. Use cmap.set_cyclic(True) or Colormap(..., cyclic=True) to suppress this warning.


DYNAMICS_FIXED_UPDATE_ORDER: True


c:\users\matts\appdata\local\programs\python\python38\lib\site-packages\sklearn\manifold\_spectral_embedding.py:245: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


Time to fit (umap): 116.34 sec


In [8]:
print(data_subdict.keys())
print(data_subdict['algos']['umap'].keys())
print(data_subdict['algos']['umap']['embedding'].shape)

dict_keys(['label', 'path', 'data', 'index', 'energies', 'num_runs', 'total_spins', 'multicell_template', 'algos'])
dict_keys(['reducer', 'embedding'])
(10000, 2)


### 2) Visualize umap

In [9]:
# 2) visualize data_subdict
plotly_express_embedding(data_subdict, 
                         color_by_index=False, 
                         as_landscape=False, 
                         fmod='jupyter', 
                         show=False, 
                         dirpath=NOTEBOOK_OUTDIR, 
                         surf=False, 
                         step=None)

# Clustering and plotting of sample points

In [10]:
from sklearn.cluster import KMeans

"""
NOTES:
- kmeans.labels_ will return [cluster_7, cluster_5, ..., cluster_0] -- cluster id for each data point
- also have: kmeans.predict([[0, 0], [12, 3]])
- also have: kmeans.cluster_centers_
"""

#kmeans_highdim = KMeans(n_clusters=8, random_state=0).fit(X)
N_CLUSTERS = 30
CLUSTER_COLOURS = {i: 'blue' for i in range(N_CLUSTERS)}  # TODO implement, currently unused
assert N_CLUSTERS <= len(CLUSTER_COLOURS.keys())

kmeans_lowdim = KMeans(n_clusters=N_CLUSTERS, random_state=0).fit(data_subdict['algos']['umap']['embedding'])

def cluster_to_colour(a):
    return CLUSTER_COLOURS[a]

In [11]:
kmeans_labels = kmeans_lowdim.labels_ 
color_vector = [cluster_to_colour(a) for a in kmeans_labels]  # TODO currently unused

clusterdata = {'color_vector': color_vector, 
               'cluster_ids': kmeans_labels,
               'order': [str(a) for a in range(N_CLUSTERS)],
               'cluster_to_idx': {a: np.where(kmeans_labels == a)[0] for a in range(N_CLUSTERS)}
               }

## Embedding with colour by cluster

In [12]:
plotly_express_embedding(data_subdict, 
                         color_by_index=False, 
                         as_landscape=False, 
                         fmod='jupyter', 
                         show=False, 
                         dirpath=NOTEBOOK_OUTDIR, 
                         surf=False, 
                         step=None,
                         clusterstyle=clusterdata)

## Plot individual points from the clustering

In [13]:
SAMPLES_PER_CLUSTER = 10

from multicell.multicell_replot import \
    replot_graph_lattice_reference_overlap_plotter, replot_modern, replot_scatter_dots

def plot_tissue_given_agg_idx(agg_index, fmod, outdir, state_int=False):
    
    # TODO do we need to load the varying simsetup_W in each dir of manyruns?
    fpath_pickle = manyruns_path + os.sep + 'multicell_template.pkl'
    with open(fpath_pickle, 'rb') as pickle_file:
        multicell = pickle.load(pickle_file)  # unpickling multicell object

    # update W as it may vary across agg indices of manyruns
    # =======================================================================
    # TODO: does manyruns/s0 correspond to agg0 in manyruns/aggregate
    # =======================================================================
    agg_datadir = manyruns_path + os.sep + 's%d' % agg_index
    W_LOAD = np.loadtxt(agg_datadir + os.sep + 'simsetup' + os.sep + 'matrix_W.txt', delimiter=',')
    #print(W_LOAD)
    multicell.matrix_W = W_LOAD
    multicell.simsetup['FIELD_SEND'] = W_LOAD
    
    # constants
    num_cells = multicell.num_cells
    simsetup = multicell.simsetup
    sidelength = int(np.sqrt(num_cells)); assert sidelength ** 2 == num_cells
    
    #smod = ''
    smod = '_last'
    #if step is not None:
    #    smod = '_%d' % step

    agg_dir = manyruns_path + os.sep + 'aggregate'
    fpath_state = agg_dir + os.sep + 'X_aggregate%s.npz' % smod
    fpath_energy = agg_dir + os.sep + 'X_energy%s.npz' % smod
    fpath_pickle = manyruns_path + os.sep + 'multicell_template.pkl'
    print(fpath_state)
    X = np.load(fpath_state)['arr_0'].T  # umap wants transpose
    X_state = X[agg_index, :]
    print(X_state.shape)

    # plot option 2) using replot
    X_state = X_state.reshape(num_cells, simsetup['N'])
    
    #outpath_ref = outdir + os.sep + 'agg%d_ref0' % agg_index
    #replot_graph_lattice_reference_overlap_plotter(
    #    X_state.T, sidelength, outpath_ref, fmod=fmod, ref_node=0)

    outpath = outdir + os.sep + 'agg%d_modern' % agg_index
    replot_modern(X_state.T, simsetup, sidelength, outpath,
                  version='3', fmod=fmod, state_int=state_int)

    outpath = outdir + os.sep + 'agg%d_scatter' % agg_index
    replot_scatter_dots(X_state.T, sidelength, outpath,
                        fmod=fmod, state_int=state_int)
    return        

In [ ]:
embedding = data_subdict['algos']['umap']['embedding']
print(kmeans_labels)
print(embedding[0,:])

for k in range(N_CLUSTERS):
    indices = clusterdata['cluster_to_idx'][k]
    num_points = min(SAMPLES_PER_CLUSTER, len(indices))
    
    cluster_outdir = NOTEBOOK_OUTDIR + os.sep + 'c%d' % k
    assert not os.path.exists(cluster_outdir)
    os.mkdir(cluster_outdir)
    
    for idx in range(num_points):
        print(indices[idx])
        agg_idx = indices[idx]
        print('plotting cluster %d, example %d (agg %d)' % (k, idx, agg_idx))
        
        # now plot agg_idx as example of cluster N
        plot_tissue_given_agg_idx(agg_idx, '', cluster_outdir)
        
        

[21 13 12 ...  2 12  0]
[2.412698   0.75344455]
4
plotting cluster 0, example 0 (agg 4)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.00_10k_periodic_R1_p3_M100_machineEps\aggregate\X_aggregate_last.npz
(900,)
20
plotting cluster 0, example 1 (agg 20)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.00_10k_periodic_R1_p3_M100_machineEps\aggregate\X_aggregate_last.npz
(900,)
31
plotting cluster 0, example 2 (agg 31)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.00_10k_periodic_R1_p3_M100_machineEps\aggregate\X_aggregate_last.npz
(900,)
42
plotting cluster 0, example 3 (agg 42)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.00_10k_periodic_R1_p3_M100_machineEps\aggregate\X_aggregate_last.npz
(900,)
44
plotting cluster 0, example 4 (agg 44)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.

19
plotting cluster 4, example 0 (agg 19)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.00_10k_periodic_R1_p3_M100_machineEps\aggregate\X_aggregate_last.npz
(900,)
21
plotting cluster 4, example 1 (agg 21)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.00_10k_periodic_R1_p3_M100_machineEps\aggregate\X_aggregate_last.npz
(900,)
49
plotting cluster 4, example 2 (agg 49)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.00_10k_periodic_R1_p3_M100_machineEps\aggregate\X_aggregate_last.npz
(900,)
117
plotting cluster 4, example 3 (agg 117)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.00_10k_periodic_R1_p3_M100_machineEps\aggregate\X_aggregate_last.npz
(900,)
129
plotting cluster 4, example 4 (agg 129)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.00_10k_periodic_R1_p3_M100_machineEps\aggr

6
plotting cluster 8, example 0 (agg 6)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.00_10k_periodic_R1_p3_M100_machineEps\aggregate\X_aggregate_last.npz
(900,)
16
plotting cluster 8, example 1 (agg 16)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.00_10k_periodic_R1_p3_M100_machineEps\aggregate\X_aggregate_last.npz
(900,)
25
plotting cluster 8, example 2 (agg 25)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.00_10k_periodic_R1_p3_M100_machineEps\aggregate\X_aggregate_last.npz
(900,)
59
plotting cluster 8, example 3 (agg 59)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.00_10k_periodic_R1_p3_M100_machineEps\aggregate\X_aggregate_last.npz
(900,)
70
plotting cluster 8, example 4 (agg 70)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.00_10k_periodic_R1_p3_M100_machineEps\aggregate\

2
plotting cluster 12, example 0 (agg 2)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.00_10k_periodic_R1_p3_M100_machineEps\aggregate\X_aggregate_last.npz
(900,)
15
plotting cluster 12, example 1 (agg 15)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.00_10k_periodic_R1_p3_M100_machineEps\aggregate\X_aggregate_last.npz
(900,)
32
plotting cluster 12, example 2 (agg 32)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.00_10k_periodic_R1_p3_M100_machineEps\aggregate\X_aggregate_last.npz
(900,)
65
plotting cluster 12, example 3 (agg 65)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.00_10k_periodic_R1_p3_M100_machineEps\aggregate\X_aggregate_last.npz
(900,)
68
plotting cluster 12, example 4 (agg 68)
I:\Development\Repositories\biomodels\celltypes\runs\multicell_manyruns\Wrandom0_gamma1.00_10k_periodic_R1_p3_M100_machineEps\aggre